### Unir dos VPCs para la comunicación de red entre VPCs

#### Problema
Necesita habilitar dos instancias en VPCs separadas para que se comuniquen entre sí de una manera simple y rentable.

#### Solución
Solicite una conexión de peering entre dos VPC, acepte la conexión de peering, actualice las tablas de rutas para cada subred de la VPC y, finalmente, pruebe la conexión de una instancia a otra.

<img src="https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781492092599/files/assets/awsc_0215.png" width="650">

In [ ]:
import boto3
import json
import time

region_aws = 'us-east-1'

ec2 = boto3.resource('ec2', region_name=region_aws)
ec2_client = boto3.client('ec2', region_name=region_aws)

In [2]:
# Create two VPCs
vpc1 = ec2.create_vpc(CidrBlock='10.10.0.0/16')
vpc1.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookVPC-1"}])
vpc2 = ec2.create_vpc(CidrBlock='10.11.0.0/16')
vpc2.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookVPC-2"}])
vpc2.wait_until_available()


# Create two subnets for each VPC
vpc1_subnet1 = ec2.create_subnet(
    CidrBlock='10.10.1.0/24', 
    VpcId=vpc1.id,
    AvailabilityZone=region_aws+'a'
)
vpc1_subnet2 = ec2.create_subnet(
    CidrBlock='10.10.2.0/24',
    VpcId=vpc1.id
)

vpc2_subnet1 = ec2.create_subnet(
    CidrBlock='10.11.1.0/24',
    VpcId=vpc2.id
)
vpc2_subnet2 = ec2.create_subnet(
    CidrBlock='10.11.2.0/24',
    VpcId=vpc2.id
)


# Create tags for subnets
vpc1_subnet1.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookVPC-1-Subnet-1"}])
vpc1_subnet2.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookVPC-1-Subnet-2"}])
vpc2_subnet1.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookVPC-2-Subnet-1"}])
vpc2_subnet2.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookVPC-2-Subnet-2"}])


# Create route tables for each VPC
vpc1_route_table = vpc1.create_route_table()
vpc1_route_table.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookVPC-1-Route-Table"}])
vpc2_route_table = vpc2.create_route_table()
vpc2_route_table.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookVPC-2-Route-Table"}])


# Associate subnets with route tables
vpc1_route_table.associate_with_subnet(SubnetId=vpc1_subnet1.id)
vpc1_route_table.associate_with_subnet(SubnetId=vpc1_subnet2.id)
vpc2_route_table.associate_with_subnet(SubnetId=vpc2_subnet1.id)
vpc2_route_table.associate_with_subnet(SubnetId=vpc2_subnet2.id)

ec2.RouteTableAssociation(id='rtbassoc-098cd028ca37dda75')

#### Crear una conexión VPC peering para conectar VPC1 con VPC2

Las conexiones de peering de VPC pueden establecerse desde una cuenta de AWS a otra cuenta de AWS. Si decide establecer conexiones de pares de VPC entre cuentas de AWS, debe asegurarse de tener la configuración correcta de IAM para crear y aceptar la conexión de pares dentro de cada cuenta. En este ejemplo será dentro de la misma cuenta.

In [3]:
vpc_peering_connection = ec2_client.create_vpc_peering_connection(
    VpcId=vpc1.id,
    PeerVpcId=vpc2.id
)

In [5]:
vpc_peering_connection_id = vpc_peering_connection['VpcPeeringConnection']['VpcPeeringConnectionId']
response = ec2_client.accept_vpc_peering_connection(VpcPeeringConnectionId=vpc_peering_connection_id)
if response['VpcPeeringConnection']['Status']['Code'] == 'active':
    print('VPC Peering Connection is active')

VPC Peering Connection is active


In [6]:
# Create a route in the route table of VPC1 to send traffic to VPC2
vpc1_route_table.create_route(
    DestinationCidrBlock=vpc2.cidr_block,
    VpcPeeringConnectionId=vpc_peering_connection_id
)

# Create a route in the route table of VPC2 to send traffic to VPC1
vpc2_route_table.create_route(
    DestinationCidrBlock=vpc1.cidr_block,
    VpcPeeringConnectionId=vpc_peering_connection_id
)

ec2.Route(route_table_id='rtb-0f59c14cc6b21e234', destination_cidr_block='10.10.0.0/16')

In [16]:
print("En la terminal ejecute el siguiente comando:")
print("    python create_ec2_ssm.py --vpc {} --subnet {}".format(vpc1.id, vpc1_subnet1.id))

En la terminal ejecute el siguiente comando:
    python create_ec2_ssm.py --vpc vpc-025e778f159d2df44 --subnet subnet-05b127a57bf739e16


In [17]:
print("También ejecute el siguiente comando:")
print("    python create_ec2_ssm.py --vpc {} --subnet {}".format(vpc2.id, vpc2_subnet1.id))

También ejecute el siguiente comando:
    python create_ec2_ssm.py --vpc vpc-08e6b3d19d517ad2c --subnet subnet-0eb66a12a1655fe9e


In [56]:
# List of the instances in the VPC1
instances = ec2.instances.filter(
    Filters=[{'Name': 'vpc-id', 'Values': [vpc1.id]}]
)
for instance in instances:
    instance1 = instance
    print("Instance in VPC1: {}".format(instance1.id))

Instance in VPC1: i-03508e373ce9ddafb


In [57]:
# List of the instances in the VPC2
instances = ec2.instances.filter(
    Filters=[{'Name': 'vpc-id', 'Values': [vpc2.id]}]
)
for instance in instances:
    instance2 = instance
    print("Instance in VPC2: {}".format(instance2.id))

Instance in VPC2: i-0940367df644f22df


In [58]:
# Select the security group of the instance
sg_1 = ec2.SecurityGroup(instance1.security_groups[0]['GroupId'])
sg_2 = ec2.SecurityGroup(instance2.security_groups[0]['GroupId'])

In [73]:
# Add an ingress rule to instance 2’s security group that allows ICMPv4 access from instance 1’s security group:
response = ec2_client.authorize_security_group_ingress(
    GroupId=sg_2.id,
    IpPermissions=[
        {
            'IpProtocol': 'icmp',
            'FromPort': -1,
            'ToPort': -1,
            'UserIdGroupPairs': [
                {
                    'GroupId': sg_1.id,
                    'Description': 'Allow ICMPv4 access'
                },
            ]
        },
    ]
)

In [74]:
print("Conectese a la instancia 1:")
print("   aws ssm start-session --target {}".format(instance1.id))

Conectese a la instancia 1:
   aws ssm start-session --target i-03508e373ce9ddafb


In [49]:
print("Desde la instancia 1 ejecute el siguiente comando: ")
print("    ping -c 4 {}".format(instance2.private_ip_address))

Desde la instancia 1 ejecute el siguiente comando: 
    ping -c 4 10.11.1.157


Puede buscar el ID de un grupo de seguridad en la consola de la VPC para mostrar todos los grupos de seguridad que hacen referencia a otros. También puede ejecutar el comando CLI `aws ec2 describe-security-group-references` para conseguirlo. Esto es útil para obtener información sobre qué grupos de seguridad hacen referencia a otros. Puede hacer referencia a grupos de seguridad en VPCs peered que pertenecen a otras cuentas de AWS pero que no se encuentran en otras regiones.

#### Discusión
Las conexiones VPC peering son no transitivas. Cada VPC necesita hacer peering con cada una de las otras VPC con las que necesita comunicarse. Este tipo de conexión es ideal cuando se tiene una VPC que aloja servicios compartidos a los que otras VPCs necesitan acceder, mientras que las VPCs "spoke" no se comunican entre sí.

Además de las conexiones de peering, es necesario configurar las tablas de rutas asociadas a las subredes de la VPC para enviar el tráfico destinado al CIDR de la VPC peered a la conexión de peering (PCX). En otras palabras, para que la VPC1 pueda comunicarse con la VPC2, la ruta de destino debe estar presente en la VPC1 y la ruta de retorno también debe estar presente en la VPC2.

Si se agrega una tercera VPC a esta receta, y se necesita que todas las VPC puedan comunicarse entre sí, habría que emparejar esa tercera VPC con las dos anteriores y actualizar todas las tablas de rutas de las VPC en consecuencia para permitir que todas las VPC se comuniquen entre sí. A medida que se añaden más VPCs a una arquitectura de red como ésta, se puede notar que el número de conexiones de peering y las actualizaciones de la tabla de rutas requeridas comienzan a aumentar exponencialmente. Debido a esto, el gateway de tránsito es una mejor opción para la comunicación transitiva de VPC utilizando tablas de rutas de gateway de tránsito.

Puede utilizar el peering de VPC de forma cruzada si es necesario, y también puede referenciar grupos de seguridad en VPCs peered de forma similar a la referencia de grupos de seguridad dentro de una única VPC. Esto le permite utilizar el mismo tipo de estrategia con la forma de administrar los grupos de seguridad en todo su entorno de AWS al utilizar el peering de VPC.

**NOTA:** La conexión de VPCs entre sí requiere rangos CIDR no superpuestos para que el enrutamiento funcione normalmente. Las tablas de rutas de la VPC deben incluir una ruta específica que dirija el tráfico destinado a la VPC conectada al peering.


#### Desafío
Las conexiones de peering de VPC pueden establecerse en todas las regiones de AWS. Conecte una VPC en otra región a la VPC que implementó en la región utilizada en el ejemplo.